In [ ]:
load Subject_A_Train.mat;
window=240; % window after stimulus (1s)

responses_train = zeros(12, 15, 240, 64, 85);
is_stimulate = zeros(12, 15, 85);

% convert to double precision
Signal=double(Signal);
Flashing=double(Flashing);
StimulusCode=double(StimulusCode);
StimulusType=double(StimulusType);

% for each character epoch
for epoch=1:size(Signal,1)
    % get reponse samples at start of each Flash
    rowcolcnt=ones(1,12);
    block = 1;
    for n=2:size(Signal,2)
        if Flashing(epoch,n)==0 && Flashing(epoch,n-1)==1
            rowcol=StimulusCode(epoch,n-1);
            responses_train(rowcol,rowcolcnt(rowcol),:,:,epoch)=Signal(epoch,n-24:n+window-25,:);
            rowcolcnt(rowcol)=rowcolcnt(rowcol)+1;
            if StimulusType(epoch, n-1) == 1
                is_stimulate(rowcol, block, epoch) = 1;               
            end
        end
        if mod(n, 504) == 0
            block = block + 1;
        end
    end
end


responses_train = reshape(mean(responses_train, 2), 12, 240, 64, 85);
is_stimulate_train = uint8(reshape(mean(is_stimulate, 2), 12, 85));

In [ ]:
clear all;  
clc;
temp=zeros(12,85,64,240);

load ('data_train_A.mat');
data = [];
label = [];

temp=permute(responses_train,[2,1,3,4]);
data_temp1=squeeze(temp(:,:,3,:));
data_temp2=squeeze(temp(:,:,5,:));
data_temp3=squeeze(temp(:,:,9,:));
data_temp4=squeeze(temp(:,:,11,:));
data_temp5=squeeze(temp(:,:,13,:));
data_temp6=squeeze(temp(:,:,22,:));
data_temp7=squeeze(temp(:,:,24,:));
data_temp8=squeeze(temp(:,:,34,:));
data_temp9=squeeze(temp(:,:,51,:));
data_temp10=squeeze(temp(:,:,56,:));
data_temp11=squeeze(temp(:,:,60,:));

responses_train=[data_temp1;data_temp2;data_temp3;data_temp4;data_temp5;data_temp6;data_temp7;data_temp8;data_temp9;data_temp10;data_temp11];


responses_train=permute(responses_train,[2,3,1]);
responses_train=reshape(responses_train,12,85,2640);
for i=1:12
    for j=1:85    
        data = [data; responses_train(i, j,:)];
        label = [label; is_stimulate_train(i, j)];
    end
end

data = reshape(data, 1020, 2640);

Model_A= fitcsvm(data,label);